This notebook exists to experiment with different methods for classification

In [1]:
%run "Feature_Selection.ipynb" 

100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:02<00:00, 73.86it/s]


In [2]:
removed_params = ['rr_std', 'RMSSD', 'pNN50', 'mean', 'lf', 'hf', 'skews', 'rr_mean', 'power_ratio', 'std', 'kurtosis', 'shannon_en'] #convert to a set?

selected_params = {}
for key in params.keys():
    if key not in removed_params:
        selected_params[key] = params[key]
print(selected_params.keys())

dict_keys(['sd_ratio'])


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
health_state = np.array(allowed_patients.get_diagnoses())

encoded_health_state = [1 if label == 'Unhealthy' else -1 for label in health_state]

print(len(health_state))

196


In [5]:
def get_balanced_accuracy(y_test, y_pred):
    """
    balanced accuracy....
    """
    num_healthy_true = np.sum([x=='Healthy' for x in y_test])
    num_unhealthy_true = len(y_test) - num_healthy_true
    count_healthy_accurate = 0
    count_unhealthy_accurate = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            count_unhealthy_accurate +=1
        elif y_pred[i] == y_test[i] == 'Healthy':
            count_healthy_accurate +=1
    healthy_percentage = count_healthy_accurate/num_healthy_true
    unhealthy_percentage = count_unhealthy_accurate/num_unhealthy_true
    balanced_accuracy = (healthy_percentage + unhealthy_percentage) * 0.5
    return balanced_accuracy

def get_specificity(y_test, y_pred):
    """
    true negative rate
    """
    true_negative = 0
    false_positive = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Healthy':
            true_negative += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
    
    return true_negative / (true_negative+false_positive)
            
def get_f1_score(y_test, y_pred):
    """
    balance between precision and recall
    """
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            true_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Unhealthy':
            false_negative += 1
    return (2*true_positive)/(2*true_positive+false_positive+false_negative)


def get_av_confusion_matrix(y_test, y_pred):
    av_confusion_mat = np.zeros(shape = (len(y_test), 2, 2))
    for i in range(0, len(y_test)):
        av_confusion_mat[i] = confusion_matrix(y_test[i], y_pred[i])
    return np.mean(av_confusion_mat, axis=0)

def scoring_function(model, X, y):
    """
    change to incorporate balanced accuracy 
    """
    
    y_pred = model.predict(X)
    y_test = y
    balanced_acc = get_balanced_accuracy(y_test, y_pred)
    specificity = get_specificity(y_test, y_pred)
    f1 = get_f1_score(y_test, y_pred)
    
    
    return f1*0.7 + balanced_acc*0.3

## SVM

In [9]:
def convert_dict_to_array(selected_params):
    no_features = len(selected_params)
    selected_params_array = np.zeros((no_channels, no_patients, no_features))
    
    for j in range(0, no_channels):
        for i, values in enumerate(selected_params.values()):
            selected_params_array[j][:, i] = values[j]
    return selected_params_array, no_features

        
selected_params_array, no_features = convert_dict_to_array(selected_params)


### Basic SVM

In [ ]:
#splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_params_array, health_state, test_size=0.3, stratify=health_state)

#init and train model, using radial basis functions
svm_model = SVC(kernel='rbf', gamma='scale', probability=True, class_weight = 'balanced')  #'scale' normalises data, prevents overfitting
#probability = True allows calculation of probabilities through 5-fold CV

svm_model.fit(X_train, y_train)

#predicting probabilities
probabilities = svm_model.predict_proba(X_test)
print(probabilities)


#predictions
y_pred = svm_model.predict(X_test)

#evaluating accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = get_balanced_accuracy(y_test, y_pred)
print("Accuracy:", accuracy)
print('balanced accuracy:', balanced_accuracy)

print(np.sum([x=='Healthy' for x in y_test]))
print(y_test, y_pred)

#view a classification report
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

### Hyper Parameter Tuning
- c is inversely proportional to l2 regularisation parameter
- gamma determines scaling of features


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score

#recall needs encoded_health_state
#splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_params_array, health_state, test_size=0.3, stratify=health_state)

# define hyperparameter grid to test
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

#initialise
svc = SVC(class_weight='balanced')

# perform grid search
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring=scoring_function)
grid_search.fit(X_train, y_train)


print(grid_search.best_params_)

# train with best parameters
best_svc = grid_search.best_estimator_
best_svc.fit(X_train, y_train)

In [ ]:
#predictions
y_pred = best_svc.predict(X_test)

#evaluating accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = get_balanced_accuracy(y_test, y_pred)

print("Accuracy:", accuracy)
print('balanced accuracy:', balanced_accuracy)

### Stratified k-fold validation

need to weight the train and test sets to cope with difference with numbers of healthy and unhealthy, stratified k-fold validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits = 3 #3 fold validation
skf = StratifiedKFold(n_splits=n_splits, shuffle=True) #can do repeated skf for better validation


accuracies=[]
balanced_accuracies=[]
y_pred_list = []
y_test_list=[]
for train_index, test_index in skf.split(selected_params_array, health_state):
    #getting test and train data sets
    X_train, X_test = selected_params_array[train_index], selected_params_array[test_index]
    y_train, y_test = health_state[train_index], health_state[test_index]
    
    y_test_list.append(y_test)
    #training model with tuned hyperparameters
    tuned_svc = grid_search.best_estimator_
    tuned_svc.fit(X_train, y_train)
    
    #evaluating model
    y_pred = tuned_svc.predict(X_test)
    y_pred_list.append(y_pred)
    accuracies.append(accuracy_score(y_test, y_pred))
    balanced_accuracies.append((get_balanced_accuracy(y_test, y_pred)))

In [ ]:
print(accuracies)
print(balanced_accuracies)

In [ ]:
print(np.mean(balanced_accuracies))

In [ ]:
print(y_pred_list[2], y_test_list[2])
print(confusion_matrix(y_pred_list[2], y_test_list[2]))

In [ ]:
print(y_pred, y_test)

In [ ]:
print("Classification Report:")
print(classification_report(y_test_list[1], y_pred_list[1]))
print(get_specificity(y_test_list[1], y_pred_list[1]))
print(get_av_confusion_matrix(y_test_list, y_pred_list))